## Web Scraping Basketball-Reference.com

### Load Selenium and Packages

In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Chrome webdriver
driver = webdriver.Chrome('/Users/alexcheng/Downloads/chromedriver')

#### Player List

In [4]:
part_1 = "/Users/alexcheng/dsi/dsi_workspace/projects/project-captsone/workspace/web_scraping/player_list/player_df_part1.csv"
part_2 = "/Users/alexcheng/dsi/dsi_workspace/projects/project-captsone/workspace/web_scraping/player_list/player_df_part2.csv"
test = "/Users/alexcheng/dsi/dsi_workspace/projects/project-captsone/workspace/web_scraping/player_list/test_players.csv"


part_1_df = pd.read_csv(part_1)
part_1_list = list(part_1_df['url'])

part_2_df = pd.read_csv(part_2)
part_2_list = list(part_2_df['url'])

test_df = pd.read_csv(test)
test_list = list(test_df['url'])

In [5]:
test_list

['http://www.basketball-reference.com/players/a/abrinal01.html',
 'http://www.basketball-reference.com/players/a/acyqu01.html',
 'http://www.basketball-reference.com/players/a/adamsst01.html',
 'http://www.basketball-reference.com/players/a/afflaar01.html',
 'http://www.basketball-reference.com/players/a/ajincal01.html',
 'http://www.basketball-reference.com/players/a/aldrico01.html']

### Web Scraping

In [4]:
tmp_list = []

In [5]:
for url in test_list:
    
    ##################################################
    ############### web scraping #####################
    ##################################################
    driver.get(url)
    
    # player name
    name = driver.find_elements_by_xpath("""//*[@id="meta"]/div[2]/h1""") 
    
    # scraping per_game table
    driver.find_element_by_xpath("""//*[@id="all_per_game"]/div[1]/div/ul/li[1]/span""").click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, """//*[@id="all_per_game"]/div[1]/div/ul/li[1]/div/ul/li[3]/button"""))).click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.CLASS_NAME, """table_outer_container""")))
    
    # capture csv text
    per_game = driver.find_element_by_id("csv_per_game")
    
    
    # scraping advanced table
    driver.find_element_by_xpath("""//*[@id="all_advanced"]/div[1]/div/ul/li[1]/span""").click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, """//*[@id="all_advanced"]/div[1]/div/ul/li[1]/div/ul/li[3]/button"""))).click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.CLASS_NAME, """table_outer_container""")))
    
    # capture csv text
    advanced = driver.find_element_by_id("csv_advanced")
    
    
    # scraping shooting table
    driver.find_element_by_xpath("""//*[@id="all_shooting"]/div[1]/div/ul/li[2]/span""").click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, """//*[@id="all_shooting"]/div[1]/div/ul/li[2]/div/ul/li[3]/button"""))).click()
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.CLASS_NAME, """table_outer_container""")))
    
    # capture csv text
    shooting = driver.find_element_by_id("csv_shooting")

    
    ##################################################
    ############### data cleaning ####################
    ##################################################
    
    # clean the player name
    for value in name:
        name = value.text
        name = str(name.encode('ascii'))
    
    # clean the per_game table
    per_game = per_game.text.encode('ascii').split()[-1][14:]
    
    per_game_list = []
    for i in per_game.split(','):
        i = float(i)
        per_game_list.append(i)
        
    # clean the advanced table
    advanced = advanced.text.encode('ascii').split()[-1][14:]
    
    advanced_list = []
    for i in advanced.split(','):
        if i == '':
            advanced_list.append(0.0)
        else:
            i = float(i)
            advanced_list.append(i)

    del advanced_list[14]
    del advanced_list[19]
    
    # clean the shooting table
    shooting = shooting.text.encode('ascii').split()[-1][14:]
    
    shooting_list = []
    for i in shooting.split(','):
        if i == '':
            shooting_list.append(0.0)
    else:
        i = float(i)
        shooting_list.append(i)

        
    ##################################################
    ############### build dataframe ##################
    ##################################################
    
    # combine and flatten dataframe
    df = [per_game_list, shooting_list, advanced_list]

    # create player stats rows to be joined to dataframe later
    player_stats = [name]
    for sublist in df:
        for val in sublist:
            player_stats.append(val)
            
    # append to tmp_list
    tmp_list.append(player_stats)

ValueError: could not convert string to float: A

#### Columns

In [ ]:
# Column names for the three tables
per_game_cols = ['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
                 '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 
                 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

shooting_cols = ['G', 'MP', 'FG%', 'AVG_DIST_FGA', '%FGA_2P', '%FGA_0-3ft', 
                 '%FGA_3-10ft','%FGA_10-16ft', '%FGA_16ft<3', '%FGA_3P', '2P%',
                 '0-3_FG%', '3-10_FG%', '10-16_FG%', '16<3_FG%', '3P%', '%ASTd_2P', 
                 '%FGA_DUNK', 'DUNKS', '%ASTd_3P', '%_CORNER3PA', '3P%_CORNER3', 
                 'HEAVE_ATT', 'HEAVE_MD']

advanced_cols = ['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
                 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
                 'WS/48', 'OBPM', 'DPM', 'BPM', 'VORP']

# Join lists together
cols = per_game_cols + shooting_cols + advanced_cols

In [ ]:
# Create dataframe
nba_player_stats = pd.DataFrame(tmp_list, columns=cols)

### Final Dataframe

In [ ]:
nba_player_stats